In [5]:
from astropy.io import fits
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import aplpy
from astropy import units as u
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
import pandas as pd
pd.set_option('display.max_columns', 100)
import seaborn as sns
sns.set(font="times",font_scale=1.7,style="ticks")
import matplotlib as mpl
mpl.rc('text', usetex=True)
import codecs
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

In [6]:
catalog = pd.read_csv("../files/cluster_search.csv")
ra,dec,name = catalog["_RA.icrs"],catalog["_DE.icrs"],catalog["Cluster"]

In [7]:
#Que_1 = '"https://gea.esac.esa.int/tap-server/tap/sync?REQUEST=doQuery&LANG=ADQL&FORMAT=csv&QUERY'\
#+'=SELECT+gaia.source_id,gaia.ra,gaia.dec,gaia.l,gaia.b,gaia.bp_rp,gaia.parallax,gaia.parallax_error,gaia.parallax_over_error,'\
#+'gaia.phot_g_mean_flux,gaia.phot_g_mean_flux_error,gaia.phot_g_mean_mag,gaia.phot_bp_rp_excess_factor,gaia.radial_velocity,gaia.radial_velocity_error,'\
#+'xmatch.original_ext_source_id,tmass.j_m,tmass.h_m,tmass.ks_m'\
#+'+FROM+gaiadr3.gaia_source+AS+gaia+INNER+JOIN+gaiadr3.tmass_psc_xsc_neighbourhood+AS+xmatch+ON+gaia.source_id=xmatch.source_id'\
#+'+INNER+JOIN+gaiadr1.tmass_original_valid+AS+tmass+ON+tmass.designation=xmatch.original_ext_source_id+WHERE+'

Que_1 = '"https://gea.esac.esa.int/tap-server/tap/sync?REQUEST=doQuery&LANG=ADQL&FORMAT=csv&QUERY='\
+'SELECT+gaia.source_id,gaia.ra,gaia.dec,gaia.l,gaia.b,gaia.phot_g_mean_mag,gaia.bp_rp,gaia.parallax,gaia.parallax_error'\
+',gaia.priam_flags,gaia.flame_flags,gaia.a_g_val,gaia.e_bp_min_rp_val,gaia.teff_val,gaia.radius_val,gaia.radius_percentile_lower,gaia.radius_percentile_upper,'\
+'xmatch.original_ext_source_id,tmass.j_m,tmass.h_m,tmass.ks_m+FROM+gaiadr2.gaia_source+AS+gaia+'\
+'INNER+JOIN+gaiadr2.tmass_best_neighbour+AS+xmatch+ON+gaia.source_id=xmatch.source_id+'\
+'INNER+JOIN+gaiadr1.tmass_original_valid+AS+tmass+ON+tmass.designation=xmatch.original_ext_source_id+WHERE+'

In [8]:
FOV_RA = 1.206 # deg
FOV_DE = 1.206 # deg
Hmag_lower = 16
sh_file = codecs.open("./download.sh",mode="w")

for N,R,D in zip(name,ra,dec):
    N = N.replace(" ","")
    ra_upper = "%.3f"%(R+0.5*FOV_RA)
    ra_lower = "%.3f"%(R-0.5*FOV_RA)
    dec_upper = "%.3f"%(D+0.5*FOV_DE)
    dec_lower = "%.3f"%(D-0.5*FOV_DE)
    RA_limit   = ra_lower+"<gaia.ra+AND+gaia.ra<"+ra_upper
    DE_limit   = dec_lower+"<gaia.dec+AND+gaia.dec<"+dec_upper
    Hmag_limit = "tmass.h_m<%.2f"%Hmag_lower 
    WHERE = "("+RA_limit+"+AND+"+DE_limit+"+AND+"+Hmag_limit+')"'
    Que="curl -o ../query/%s.csv "%(N)+Que_1+WHERE
    print(Que,file=sh_file)